In [1]:
import pandas as pd
df=pd.read_csv(r"CaliHouse.csv")
df

FileNotFoundError: [Errno 2] No such file or directory: 'CaliHouse.csv'

In [ ]:
df2=df.dropna(subset=['title'])[['title','selftext']]
df2 #leave it to anaylze later

In [ ]:
job = df2[df2['selftext'].str.contains("full time job")|df['title'].str.contains("full time job")]
job.to_csv('job.csv',index = True)

In [ ]:
credit = df2[df2['selftext'].str.contains("credit card")|df['title'].str.contains("credit card")]
credit.to_csv('credit.csv',index = True)

In [ ]:
dt = df2[df2['selftext'].str.contains("don't know")|df['title'].str.contains("don't know")]
dt.to_csv('dt.csv',index = True)

In [ ]:
#Take interesting columns
df1=df[['selftext','num_comments','score','title']]
#drop those with text=NA
df1=df1.dropna(subset=['selftext'])
#drop meaningless text
df1=df1[(df1.selftext!='[removed]') & (df1.selftext!='[deleted]')].reset_index()
#drop duplicate
duplicate = df1.duplicated()
print('There are', duplicate.sum(), 'duplicated values.')
df1=df1.drop_duplicates()
len(df1)

In [ ]:
Markov=df1['selftext']

In [ ]:
full_time = df1[df1['selftext'].str.contains("full time job")|df['title'].str.contains("full time job")]
full_time.to_csv('full_time_job.csv',index = True)

In [ ]:
dont_know = df1[df1['selftext'].str.contains("dont know")|df['title'].str.contains("dont know")]
dont_know.to_csv('dont_know.csv',index = True)

In [ ]:
credit = df1[df1['selftext'].str.contains("credit card")|df['title'].str.contains("credit card")]
credit.to_csv('credit_card.csv',index = True)

### Convert to lowercase

In [ ]:
df1['selftext']=df1['selftext'].str.lower()
df1['selftext']

### Remove emoji

In [ ]:
def remove_emoji(text):
    text = text.encode('ascii', 'ignore').decode()
    return text

In [ ]:
df1['selftext']=df1['selftext'].apply(lambda x: remove_emoji(x))
df1['selftext'][0]

### Remove url, punctuation, and special characters

In [ ]:
#Using regular expression
import re
def remove_other(x):
    x = re.sub("\$"," ", x) #remove $
    x = re.sub("https*\S+", " ", x) #remove url
    x = re.sub("\'\w+", '', x) #remove i'm,we're,let's after the '
    x = re.sub("[0-9]+", '', x) #remove numbers
    return x

In [ ]:
df1['selftext']=df1['selftext'].apply(lambda x: remove_other(x))
df1['selftext'][0]

### Remove stopwords

In [ ]:
import nltk
from nltk.corpus import stopwords
#nltk.download("stopwords") #uncomment it when run it for the first time
stop_words = set(stopwords.words("english"))
print(stop_words) #all preloaded stopwords

In [ ]:
def remove_stopwords(text):
    text = " ".join([word for word in text.split() if word not in stop_words])
    return text

In [ ]:
df1['selftext']=df1['selftext'].apply(lambda x: remove_stopwords(x))
df1['selftext'][0]

In [ ]:
spec_chars = ["!",'“','"',"#","%","&","'","(",")",
              "*","+",",","-",".","/",":",";","<",
              "=",">","?","@","[","\\","]","^","_",
              "`","{","|","}","~","–"]
for char in spec_chars:
    df1['selftext'] = df1['selftext'].str.replace(char, ' ')
    df1['selftext'] = df1['selftext'].str.split().str.join(" ")
df1['selftext']

### Check the resulting text to see whatelse we need to clean

In [ ]:
df1['selftext'][4]

### Example of Lemmatization(change rules to rule)

In [ ]:
import nltk
#nltk.download('wordnet') #uncommented for 1st time running
from nltk.stem import WordNetLemmatizer

words=df1['selftext'][4].split()
lemmatizer = WordNetLemmatizer()
for word in words:
    print(word + " ---> " + lemmatizer.lemmatize(word))

In [ ]:
def lemmatize(words):
    lemmatizer = WordNetLemmatizer()
    nwords=[]
    for word in words:
        word=lemmatizer.lemmatize(word)
        nwords.append(word)
    return ' '.join(nwords)

In [ ]:
df1['selftext']= df1['selftext'].str.split().apply(lambda x: lemmatize(x))
df1['selftext'][4]

### Split words

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vec=CountVectorizer()
counts=vec.fit_transform(df1['selftext'])
counts=counts.toarray()
count_df=pd.DataFrame(counts,columns=vec.get_feature_names())
big_df=pd.concat((df1,count_df),axis=1)
big_df.head()

In [ ]:
# List of all words across rows
import itertools
import collections
def count_words(text):
    all_words = list(itertools.chain(*text.str.split()))
    counts = collections.Counter(all_words)
    counts_df = pd.DataFrame(counts.most_common(100),
                            columns=['words', 'count'])

    return counts_df
count_words(df1['selftext'])

In [ ]:
stop_words.update({'would','k','im','could','also','amp','much','one','like','get','since'})
df1['selftext']=df1['selftext'].apply(lambda x: remove_stopwords(x))
count_df=count_words(df1['selftext'])
count_df.head(30)

In [ ]:
import plotly.express as px
fig = px.bar(count_df.head(10), x='words', y='count', title="Top 10 words in Adult")
fig.show()

In [ ]:
!pip install wordcloud

In [ ]:
# Libraries

from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Create a list of word
text=' '.join(count_df.words)

# Create the wordcloud object
wordcloud = WordCloud(width=1500, height=1000, margin=0,background_color='white').generate(text)

# Display the generated image:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.margins(x=0, y=0)
plt.show()

### Two grams Frequency

In [ ]:
from nltk.util import ngrams
def count_2gram(text):
    all_words = list(itertools.chain(*text.str.split()))
    esBigrams = ngrams(all_words, 2)
    counts = collections.Counter(esBigrams)
    counts_df = pd.DataFrame(counts.most_common(100),
                            columns=['words', 'count'])
    dictionary2 = [' '.join(tup) for tup in counts_df.words]
    counts_df.words=dictionary2

    return counts_df
counts_df2=count_2gram(df1['selftext'])
counts_df2.head(20)

In [ ]:
fig = px.bar(counts_df2.head(10), x='words', y='count', title="Top 10 2-grams in Adult")
fig.show()

In [ ]:
words_dict = dict(counts_df2.values)

wordCloud = WordCloud(max_words=200, height=1000, width=1500)
wordCloud.generate_from_frequencies(words_dict)
plt.title('Most frequently occurring bigrams connected by same colour and font size')
plt.imshow(wordCloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
from nltk.util import ngrams
def count_3gram(text):
    all_words = list(itertools.chain(*text.str.split()))
    esBigrams = ngrams(all_words, 3)
    counts = collections.Counter(esBigrams)
    counts_df = pd.DataFrame(counts.most_common(100),
                            columns=['words', 'count'])
    dictionary3 = [' '.join(tup) for tup in counts_df.words]
    counts_df.words=dictionary3

    return counts_df
counts_df3=count_3gram(df1['selftext'])
counts_df3.head(20)

In [ ]:
fig = px.bar(counts_df3.head(10), x='words', y='count', title="Top 10 3-grams in Adult")
fig.show()

In [ ]:
from nltk.util import ngrams
def count_4gram(text):
    all_words = list(itertools.chain(*text.str.split()))
    esBigrams = ngrams(all_words, 4)
    counts = collections.Counter(esBigrams)
    counts_df = pd.DataFrame(counts.most_common(100),
                            columns=['words', 'count'])
    dictionary4 = [' '.join(tup) for tup in counts_df.words]
    counts_df.words=dictionary4

    return counts_df
counts_df4=count_4gram(df1['selftext'])
counts_df4.head(20)

In [ ]:
fig = px.bar(counts_df4.head(10), x='words', y='count', title="Top 10 4-grams in Adult")
fig.show()

### Try Markov Chain to generate synthetic text

In [ ]:
import random
def markov_text(s, n, seed,length):
    '''
    Generates synthetic text according to an n-th order Markov model
    Parameter s: input string of real text
    Parameter n: integer n, the order of the model
    Parameter seed: the initial string that gets the Markov model started.
    Parameter length: integer, the size of the text to generate. default=100
    '''
    counts = count_ngrams(s, n+1)
    fake=seed
    for i in range(length):
        previous = fake[(-n):]
        # filter dict to keep only matching grams
        sub = {}
        for key in counts:
            if key[:-1] == previous: 
                sub[key] = counts[key]
    # convert to lists for use with random.choices
        choices = list(sub.keys())
        weights = [sub[key] for key in choices]
        new_gram = random.choices(choices, weights)[0]
        new_char = new_gram[-1]
        fake+=new_char
    return fake 

In [ ]:
def count_ngrams(s,n=1):
    '''
    Counts the number of times each n-gram occurs in a string 's'
    Parameter s: input string
    Parameter n: input integer n to specify 'n-gram' that need to be count
    '''
    D={}
    #loop through s, untill there are not enough characters left forming n-gram
    for i in range(len(s)-(n-1)): 
        D[s[i:i+n]]=0 # initialize values to avoid key errors
    for i in range(len(s)-(n-1)):
        D[s[i:i+n]]+=1 #the value of each n-gram is their counts in s
    return D
    
#test the output
count_ngrams("tortoise", n = 2)

In [ ]:
import itertools
all_words = list(itertools.chain(*Markov.str.split()))
s=' '.join(all_words)

In [ ]:
markov_text(s, n = 9, length = 400, seed = "full time")